In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers
!pip install datasets
!huggingface-cli login

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.1 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://hugg

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import pandas as pd
import os
from tqdm import tqdm
import traceback

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load MBPP dataset
ds = load_dataset("google-research-datasets/mbpp", "full")
test_data = ds["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

...

prompt-00000-of-00001.parquet:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/374 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

# Debugging, testing on single row

In [ ]:
# For example, select a single sample from the MBPP test dataset.
# (Adjust the index as needed.)
sample = test_data[1]
prompt = sample["text"]       # The task prompt (or question)
reference = sample["code"]    # The reference solution/code
test_list = sample.get("test_list", [])  # List of tests (if present)

# Optional: Print these out for confirmation.
print("=== Defined Variables ===")
print("Task ID:", sample["task_id"])
print("Prompt:\n", prompt)
print("\nReference Code:\n", reference)
print("\nTest List:", test_list)

=== Defined Variables ===
Task ID: 12
Prompt:
 Write a function to sort a given matrix in ascending order according to the sum of its rows.

Reference Code:
 def sort_matrix(M):
    result = sorted(M, key=sum)
    return result

Test List: ['assert sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])==[[1, 1, 1], [1, 2, 3], [2, 4, 5]]', 'assert sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1]])==[[-2, 4, -5], [1, -1, 1], [1, 2, 3]]', 'assert sort_matrix([[5,8,9],[6,4,3],[2,1,4]])==[[2, 1, 4], [6, 4, 3], [5, 8, 9]]']


# Improved Iteration Testing

# With Pass@k Interpretation AND CHAIN OF THOUGHT

In [ ]:
import re
import traceback
import os
import json
import time
import math
import torch
import ast
import signal
from contextlib import contextmanager
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm

@contextmanager
def time_limit(seconds):
    """
    Raises TimeoutError if the code block inside the context
    takes more than `seconds` to execute.
    """
    def signal_handler(signum, frame):
        raise TimeoutError("Code execution timed out.")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


def truncate_to_function_body(snippet: str, func_name: str) -> str:
    """
    Given a code snippet (triple-backtick block) and a function name (func_name),
    try to extract just the definition of `def func_name(...):` plus its body lines,
    ignoring any trailing lines that are outside or after the function.

    Raises ValueError if we can't find such a function or still can't parse it.
    """
    # Find the line that says: def <func_name>(...):
    pattern = rf"^(\s*)def\s+{re.escape(func_name)}\s*\(.*?\):"
    lines = snippet.splitlines()
    func_start_idx = None
    base_indent = None

    for idx, line in enumerate(lines):
        m = re.match(pattern, line)
        if m:
            func_start_idx = idx
            # The indentation level of the 'def' line
            base_indent = len(m.group(1).replace('\t', '    '))
            break

    if func_start_idx is None:
        raise ValueError(f"Could not find 'def {func_name}(...):' in snippet.")

    # Collect lines from func_start_idx forward while they are indented or blank.
    # Stop if we see another top-level "def" with the same or lesser indentation
    # or if we run out of lines.
    collected = []
    collected.append(lines[func_start_idx])

    for i in range(func_start_idx + 1, len(lines)):
        line = lines[i]
        # If we see a top-level "def" or class at the same indentation, stop:
        #    e.g. ^(\s*)def ...
        next_def_m = re.match(r"^(\s*)def\s+\w+\s*\(.*?\):", line)
        if next_def_m:
            # Compare indentation
            next_def_indent = len(next_def_m.group(1).replace('\t','    '))
            if next_def_indent <= base_indent:
                # It's a new top-level function -> stop collecting
                break

        # If it looks like a statement with indentation at least base_indent+4,
        # or it might be blank, we keep it. If it's unindented or too shallow, we stop.
        # We'll do a small check: actual indentation is number of leading spaces
        leading_spaces = len(line) - len(line.lstrip(' '))  # only counting spaces, ignoring tabs
        if line.strip() == '':
            # Blank lines are allowed
            collected.append(line)
        elif leading_spaces > base_indent:
            collected.append(line)
        else:
            # We found a line that is not sufficiently indented -> top-level code or code for another function
            break

    truncated_snippet = "\n".join(collected)

    # Test parse
    try:
        ast.parse(truncated_snippet)
    except SyntaxError as e:
        raise ValueError(f"Truncated function body still not parseable: {e}")

    return truncated_snippet


def fallback_extract_definition(generated_text: str, func_name: str) -> str:
    """
    Fallback: If no triple-backtick code blocks are valid, attempt to extract
    a function definition matching `def <func_name>( ... ):` from the *raw*
    generation, plus its body lines (until next top-level 'def' or no more lines).

    Raises ValueError if not found or not parseable.
    """
    pattern = rf"(def\s+{re.escape(func_name)}\s*\(.*?\):)"
    match = re.search(pattern, generated_text)
    if not match:
        raise ValueError(f"No 'def {func_name}(...)' line found in fallback search.")

    start_idx = match.start()
    # We'll parse from that line forward:
    relevant_text = generated_text[start_idx:]

    # Now do the same approach as `truncate_to_function_body` but on relevant_text
    # because we might have trailing code that breaks parse.
    truncated_snippet = truncate_to_function_body(relevant_text, func_name)
    return truncated_snippet


def extract_valid_code_block(
    generated_text: str,
    fallback_func_name: str = ""
) -> str:
    """
    Extracts *all* triple-backtick code blocks from the text.
    For each block in order:
      1) Try `ast.parse(block)` directly.
      2) If that fails, look for the function named fallback_func_name in that block,
         extract just that function's body, try parse again.
      3) If that also fails, skip and move to the next code block.

    If no block is valid, we try the fallback approach on the *raw* text:
      - Searching for `def <fallback_func_name>(...):` and isolating its body.

    Raises ValueError if nothing works.
    """
    pattern = r"```(.*?)```"
    blocks = re.findall(pattern, generated_text, flags=re.DOTALL)

    if blocks:
        for block in blocks:
            candidate = block.strip()

            # Some model outputs start with "python", "bash", etc. on the first line
            lines = candidate.split("\n", 1)
            if len(lines) == 2:
                first_line = lines[0].lower().strip()
                if first_line in ["python", "python3", "bash", "shell", "sql", "sh"]:
                    candidate = lines[1].strip()

            # Skip empty or pure language name blocks
            lower_candidate = candidate.lower()
            if not candidate or lower_candidate in ["python", "python3", "bash", "shell", "sql", "sh"]:
                continue

            # 1) Try parse as-is
            try:
                ast.parse(candidate)
                # If parse works, we accept it
                return candidate
            except SyntaxError:
                # 2) If there's an expected function name, try to isolate it from that snippet
                if fallback_func_name:
                    try:
                        truncated = truncate_to_function_body(candidate, fallback_func_name)
                        return truncated
                    except ValueError:
                        pass
                # If that also fails, we move on to next block

    # If we get here, no code block parsed successfully
    # Attempt the fallback approach in the *full* generated_text
    if fallback_func_name:
        try:
            fallback_snippet = fallback_extract_definition(generated_text, fallback_func_name)
            return fallback_snippet
        except ValueError as e:
            raise ValueError(f"No parseable triple-backtick block, fallback also failed. Reason: {e}")
    else:
        raise ValueError("No parseable triple-backtick block found, no fallback function name provided.")


def extract_expected_signature(text):
    """
    Extracts the function name and parameters from a Python function signature.
    Example: "def first_repeated_char(str1): ..." returns ("first_repeated_char", "str1")
    """
    pattern = r"def\s+(\w+)\s*\((.*?)\):"
    match = re.search(pattern, text)
    if match:
        return match.group(1), match.group(2)
    return None, None


def run_tests_on_extracted_code(expected_func_name, generated_text, test_list):
    """
    Executes the extracted code and runs test cases.
    """
    local_namespace = {}
    test_details = []

    # 1) Extract
    try:
        code_to_execute = extract_valid_code_block(generated_text, fallback_func_name=expected_func_name)
    except ValueError as e:
        error_msg = f"No valid code block found: {str(e)}"
        return False, [False]*len(test_list), error_msg, []

    print("=== DEBUG: Extracted/Chosen code block (or fallback) ===")
    print(code_to_execute)
    print("=========================================================")

    # 2) We already know it likely parses, but do final parse check:
    try:
        ast.parse(code_to_execute)
    except SyntaxError as e:
        error_msg = f"Code execution error (syntax invalid): {e}\n{traceback.format_exc()}"
        return False, [False]*len(test_list), error_msg, []

    # 3) Execute with time limit
    try:
        with time_limit(5):
            exec(code_to_execute, {}, local_namespace)
    except TimeoutError as e:
        error_msg = f"Code execution error (timeout): {e}"
        return False, [False]*len(test_list), error_msg, []
    except Exception as e:
        error_msg = f"Code execution error: {e}\n{traceback.format_exc()}"
        return False, [False]*len(test_list), error_msg, []

    # 4) If the function name is missing, try to rename from the single callable
    if expected_func_name and expected_func_name not in local_namespace:
        callables = [(n, obj) for n, obj in local_namespace.items() if callable(obj)]
        if len(callables) == 1:
            candidate_name, candidate_obj = callables[0]
            local_namespace[expected_func_name] = candidate_obj
        else:
            # If multiple callables or zero, no easy fix
            for n, obj in callables:
                if expected_func_name.lower() in n.lower() or n.lower() in expected_func_name.lower():
                    local_namespace[expected_func_name] = obj
                    break

    # 5) Evaluate each test
    test_pass_list = []
    for test_code in test_list:
        info = {
            "test": test_code,
            "passed": False,
            "expected": None,
            "actual": None,
            "error": None
        }

        # If it's `assert X == Y`
        m = re.match(r'assert\s+(.+?)\s*==\s*(.+)', test_code)
        if m:
            lhs_expr = m.group(1).strip()
            rhs_expr = m.group(2).strip()
            try:
                with time_limit(5):
                    lhs_val = eval(lhs_expr, {}, local_namespace)
                    rhs_val = eval(rhs_expr, {}, local_namespace)
                info["actual"] = lhs_val
                info["expected"] = rhs_val
                if lhs_val == rhs_val:
                    info["passed"] = True
                else:
                    info["error"] = (
                        f"AssertionError: {lhs_expr} = {lhs_val} != {rhs_expr} = {rhs_val}"
                    )
            except TimeoutError:
                info["error"] = "Test execution timed out."
            except Exception as e:
                info["error"] = f"Error evaluating expressions: {e}"
        else:
            # Possibly "assert condition" or some snippet
            try:
                with time_limit(5):
                    exec(test_code, {}, local_namespace)
                info["passed"] = True
            except TimeoutError:
                info["error"] = "Test execution timed out."
            except AssertionError as e:
                info["error"] = f"AssertionError: {str(e)}"
            except Exception as e:
                info["error"] = f"Exception during test: {e}"

        test_pass_list.append(info["passed"])
        test_details.append(info)

    all_passed = all(test_pass_list)
    return all_passed, test_pass_list, "", test_details


# ---------------------- Main code + model inference ---------------------------

# Updated system prompt for Chain of Thought
system_prompt = (
    "You are a Python code generator who reasons step-by-step.\n"
    "First, analyze the problem carefully to ensure you understand the requirements.\n"
    "Second, clearly explain your approach using mathematical reasoning and algorithmic thinking.\n"
    "Third, outline the key steps in your algorithm before writing any code.\n"
    "Finally, implement EXACTLY ONE SOLUTION in a SINGLE code block marked with triple backticks.\n\n"
    "IMPORTANT RULES:\n"
    "1. NEVER include multiple code blocks - only provide ONE final solution inside triple backticks.\n"
    "2. ALWAYS use the EXACT function signature provided in the prompt.\n"
    "3. DO NOT use input() functions - read all data from function parameters.\n"
    "4. Ensure your solution correctly handles all test cases and edge cases.\n"
    "5. After reasoning, provide ONLY the implementation without any additional text within the code block."
)

def main():
    # Load your model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
    model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # Load MBPP dataset
    ds = load_dataset("google-research-datasets/mbpp", "full")
    test_data = ds["test"]

    # Setup output directory for Google Colab
    from google.colab import drive
    drive.mount('/content/drive')
    output_dir = "//content/drive/MyDrive/503/Llama_cot"
    os.makedirs(output_dir, exist_ok=True)

    results = []
    batch_size = 10

    pass_at_k_totals = {1: [], 5: [], 10: []}
    tpr_list = []

    total_gen_time = 0.0
    total_tokens_generated = 0
    num_samples = 10  # number of samples per task

    # If CUDA, measure memory before
    if torch.cuda.is_available():
        mem_before = torch.cuda.memory_allocated(device)
    else:
        mem_before = None

    # For demonstration, process the first N tasks, REMOVE THE RANGE TO RUN THE ENTIRE DATASET
    test_samples = test_data.select(range(50))

    for idx, sample in enumerate(tqdm(test_samples, desc="Processing test samples")):
        prompt = sample["text"]
        reference = sample["code"]
        test_list = sample.get("test_list", [])

        # Extract function signature from reference
        expected_func_name, expected_params = extract_expected_signature(reference)
        if expected_func_name:
            expected_signature_str = f"def {expected_func_name}({expected_params}):"
        else:
            expected_signature_str = ""

        # Build user prompt with Chain of Thought guidance
        user_prompt = (
            f"Task:\n{prompt}\n"
            f"Required function signature:\n{expected_signature_str}\n\n"
            "Think step-by-step:\n"
            "1. What is this problem asking for?\n"
            "2. What are the key steps or algorithms needed?\n"
            "3. Are there any edge cases to consider?\n"
            "4. Implement your complete solution inside triple backticks."
        )

        # Combine strict system prompt + user prompt
        full_prompt = f"{system_prompt}\n\nUser request:\n{user_prompt}"

        # Generate code samples with temperature settings to balance creativity and correctness
        inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
        generation_config = dict(
            max_new_tokens=1024,  # Increased token limit for CoT reasoning
            temperature=0.6,      # Reduced temperature for more focused solutions
            top_p=0.95,           # Slightly reduced top_p for more predictable outputs
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            num_return_sequences=num_samples,
        )

        start_time = time.time()
        with torch.no_grad():
            output_ids = model.generate(**inputs, **generation_config)
        gen_time = time.time() - start_time
        total_gen_time += gen_time

        # Decode the multiple outputs
        if num_samples == 1:
            predictions = [tokenizer.decode(output_ids[0], skip_special_tokens=True)]
        else:
            predictions = [
                tokenizer.decode(ids, skip_special_tokens=True)
                for ids in output_ids
            ]

        # Count tokens
        total_tokens_generated += sum(len(tokenizer.tokenize(pred)) for pred in predictions)

        # Save complete reasoning to file for analysis
        reasoning_dir = os.path.join(output_dir, "chain_of_thought")
        os.makedirs(reasoning_dir, exist_ok=True)
        for i, pred in enumerate(predictions):
            cot_path = os.path.join(reasoning_dir, f"task_{sample['task_id']}_reasoning_{i}.txt")
            with open(cot_path, "w") as f:
                f.write(pred)

        # Evaluate each prediction
        correct_flags = []
        tpr_scores = []
        prediction_details = []

        for pred in predictions:
            all_passed, pass_list, load_error, test_detail = run_tests_on_extracted_code(
                expected_func_name, pred, test_list
            )
            correct_flags.append(all_passed)
            tpr = sum(pass_list) / len(pass_list) if pass_list else 0.0
            tpr_scores.append(tpr)
            prediction_details.append({
                "prediction": pred,
                "all_passed": all_passed,
                "pass_list": pass_list,
                "test_details": test_detail,
                "load_error": load_error
            })

        c = sum(correct_flags)
        n = num_samples

        # Compute pass@k
        task_pass_at_k = {}
        for k in [1, 5, 10]:
            if n < k:
                task_pass_at_k[k] = 1.0 if c > 0 else 0.0
            else:
                try:
                    comb_n = math.comb(n, k)
                    comb_n_minus_c = math.comb(n - c, k) if (n - c) >= k else 0
                    pass_at_k_value = 1 - comb_n_minus_c / comb_n
                except Exception:
                    pass_at_k_value = 0.0
                task_pass_at_k[k] = pass_at_k_value

        best_tpr = max(tpr_scores) if tpr_scores else 0.0
        tpr_list.append(best_tpr)
        for k in [1, 5, 10]:
            pass_at_k_totals[k].append(task_pass_at_k[k])

        best_idx = tpr_scores.index(best_tpr) if tpr_scores else 0
        best_pred = prediction_details[best_idx] if prediction_details else {}

        # Gather results
        results.append({
            "task_id": sample["task_id"],
            "prompt": prompt,
            "reference": reference,
            "expected_signature": expected_signature_str,
            "final_prompt": full_prompt,
            "best_prediction": best_pred.get("prediction", ""),
            "all_tests_passed": int(best_pred.get("all_passed", False)),
            "individual_test_results": best_pred.get("pass_list", []),
            "test_details": json.dumps(best_pred.get("test_details", []), indent=2),
            "load_error": best_pred.get("load_error", ""),
            "correct_count": c,
            "tpr": best_tpr,
            "pass@1": task_pass_at_k[1],
            "pass@5": task_pass_at_k[5],
            "pass@10": task_pass_at_k[10],
            "generation_time": gen_time,
            "num_tokens": sum(len(tokenizer.tokenize(pred)) for pred in predictions),
        })

        # Save partial CSV every batch_size tasks
        if (idx + 1) % batch_size == 0:
            df_batch = pd.DataFrame(results)
            batch_csv_path = os.path.join(output_dir, f"cot_results_{idx + 1}.csv")
            df_batch.to_csv(batch_csv_path, index=False)
            print(f"Saved batch results to {batch_csv_path}")

    # Save final CSV
    df_final = pd.DataFrame(results)
    final_csv_path = os.path.join(output_dir, f"cot_results_final_{len(test_samples)}.csv")
    df_final.to_csv(final_csv_path, index=False)
    print(f"Saved final results to {final_csv_path}")

    # Compute overall metrics
    overall_pass_at_1 = (
        sum(pass_at_k_totals[1]) / len(pass_at_k_totals[1])
        if pass_at_k_totals[1] else 0.0
    )
    overall_pass_at_5 = (
        sum(pass_at_k_totals[5]) / len(pass_at_k_totals[5])
        if pass_at_k_totals[5] else 0.0
    )
    overall_pass_at_10 = (
        sum(pass_at_k_totals[10]) / len(pass_at_k_totals[10])
        if pass_at_k_totals[10] else 0.0
    )
    atpr = sum(tpr_list) / len(tpr_list) if tpr_list else 0.0
    throughput = total_tokens_generated / total_gen_time if total_gen_time > 0 else 0.0

    # Memory overhead if CUDA
    if torch.cuda.is_available():
        mem_after = torch.cuda.memory_allocated(device)
        memory_overhead = mem_after - mem_before if mem_before is not None else None
    else:
        memory_overhead = None

    print("=== Metrics Summary ===")
    print(f"Pass@1: {overall_pass_at_1:.2f}")
    print(f"Pass@5: {overall_pass_at_5:.2f}")
    print(f"Pass@10: {overall_pass_at_10:.2f}")
    print(f"Average Test Pass Rate (ATPR): {atpr:.2f}")
    print(f"Throughput (tokens/sec): {throughput:.2f}")
    if memory_overhead is not None:
        print(f"Memory Overhead (bytes): {memory_overhead}")

    # Save summary
    summary_path = os.path.join(output_dir, f"cot_summary_{len(test_samples)}.txt")
    with open(summary_path, "w") as f:
        f.write("=== Chain of Thought Metrics Summary ===\n")
        f.write(f"Pass@1: {overall_pass_at_1:.2f}\n")
        f.write(f"Pass@5: {overall_pass_at_5:.2f}\n")
        f.write(f"Pass@10: {overall_pass_at_10:.2f}\n")
        f.write(f"Average Test Pass Rate (ATPR): {atpr:.2f}\n")
        f.write(f"Throughput (tokens/sec): {throughput:.2f}\n")
        if memory_overhead is not None:
            f.write(f"Memory Overhead (bytes): {memory_overhead}\n")
    print(f"Summary saved to {summary_path}")


if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Processing test samples:   2%|▏         | 1/50 [00:23<18:53, 23.13s/it]

=== DEBUG: Extracted/Chosen code block (or fallback) ===
result = []
=== DEBUG: Extracted/Chosen code block (or fallback) ===
def remove_Occ(s, ch):
    i = 0
    while i < len(s):
        if s[i] == ch:
            i += 1
        else:
            s = s[:i] + s[i+1:]
    return s
=== DEBUG: Extracted/Chosen code block (or fallback) ===
def remove_Occ(s, ch):
    n = len(s)
    first_occurrence = s.find(ch)
    last_occurrence = s.rfind(ch)
    
    if first_occurrence!= -1 and last_occurrence!= -1:
        return s[:first_occurrence] + s[last_occurrence + 1:]
    else:
        return s
=== DEBUG: Extracted/Chosen code block (or fallback) ===
def remove_Occ(s,ch):
    """
    Removes the first and last occurrence of a character from the string.

    Parameters:
    s (str): The input string.
    ch (str): The character to be removed.

    Returns:
    str: The string with the first and last occurrence of the character removed.
    """
    # Initialize an empty string to store the resul

Processing test samples:   4%|▍         | 2/50 [00:45<18:20, 22.92s/it]

=== DEBUG: Extracted/Chosen code block (or fallback) ===
def sort_matrix(M):
    """
    Sorts a given matrix in ascending order according to the sum of its rows.

    Args:
        M (list of lists): The input matrix.

    Returns:
        list of lists: The sorted matrix.

    Raises:
        ValueError: If the input matrix is empty or not a list of lists.
    """
    # Check if input is a list of lists
    if not isinstance(M, list) or not all(isinstance(row, list) for row in M):
        raise ValueError("Input must be a list of lists.")

    # Check if input is empty
    if not M:
        raise ValueError("Input matrix cannot be empty.")

    # Calculate the sum of each row
    row_sums = [sum(row) for row in M]

    # Sort the rows based on their sums
    sorted_rows = sorted(zip(row_sums, M), key=lambda x: x[0])

    # Reconstruct the sorted matrix
    sorted_matrix = [row for row, _ in sorted_rows]

    return sorted_matrix
=== DEBUG: Extracted/Chosen code block (or fallback) ==

...

=== DEBUG: Extracted/Chosen code block (or fallback) ===
def opposite_Signs(x,y):
    # Step 1: Determine the signs of the two numbers
    sign_x = 'positive' if x > 0 else 'negative'
    sign_y = 'positive' if y > 0 else 'negative'

    # Step 2: Compare the signs of the two numbers
    if sign_x == sign_y:
        return 'No Signs'
    elif sign_x == 'positive' and sign_y == 'negative':
        return 'Opposite Signs'
    else:
        return 'Same Signs'

# Test cases
print(opposite_Signs(5, 3))   # Expected output: Opposite Signs
print(opposite_Signs(-5, 3))  # Expected output: Same Signs
print(opposite_Signs(5, -3))  # Expected output: Same Signs
print(opposite_Signs(-5, -3)) # Expected output: Opposite Signs
No Signs
Same Signs
Opposite Signs
No Signs
=== DEBUG: Extracted/Chosen code block (or fallback) ===
def opposite_Signs(x,y):
    if x > 0 and y > 0:
        return False
    elif x < 0 and y < 0:
        return False
    else:
        return True
=== DEBUG: Extracted/Chosen 

Processing test samples:  98%|█████████▊| 49/50 [21:12<00:22, 22.29s/it]

=== DEBUG: Extracted/Chosen code block (or fallback) ===
def is_octagonal(n):
    """
    Checks if a number is an nth octagonal number.

    Args:
        n (int): The number of the octagonal number.

    Returns:
        bool: True if the number is an nth octagonal number, False otherwise.
    """
    # Edge cases
    if n == 0:
        return False
    if n == 1:
        return False

    # Calculate the nth octagonal number using the derived formula
    nth_octagonal = 2 * n**2 - n

    # Check if the number is an nth octagonal number
    return nth_octagonal == n**2
=== DEBUG: Extracted/Chosen code block (or fallback) ===
# Define a list to store the first n octagonal numbers
octs = [0]*(n+1)

# Initialize the first octagonal number
octs[1] = 1

# Calculate the nth octagonal number
for i in range(2, n+1):
    # Calculate the current octagonal number using the formula: (8i-1)/2
    octs[i] = (8*i-1)//2

# Return the nth octagonal number
return octs[n]

# Test the function
print(is_

Processing test samples: 100%|██████████| 50/50 [21:34<00:00, 25.89s/it]

=== DEBUG: Extracted/Chosen code block (or fallback) ===
max_length = 0
prev_diff = 0
for i in range(n):
    diff = arr[i] - arr[i-1]
    if diff > max_length:
        max_length = diff
    if diff > prev_diff:
        prev_diff = diff
    if i == n - 1:
        return max_length
=== DEBUG: Extracted/Chosen code block (or fallback) ===
def max_len_sub(arr, n):
    dp = [0] * n
    dp[0] = arr[0]
    dp[1] = abs(arr[0] - arr[1])
    for i in range(2, n):
        dp[i] = max(dp[i - 1], dp[i - 2] + abs(arr[i] - arr[i - 1]))
    return dp[-1]
=== DEBUG: Extracted/Chosen code block (or fallback) ===
def max_len_sub(arr, n):
    dp = [0] * n
    dp[0] = 0
    for i in range(1, n):
        dp[i] = 1 + dp[i-1] if arr[i] - arr[i-1] > 0 else 1
    return max(dp)
=== DEBUG: Extracted/Chosen code block (or fallback) ===
def max_len_sub(arr, n):
    # Step 1: Initialize variables to store the maximum length and current window sum
    max_len = 0
    window_sum = 0
    
    # Step 2: Iterate over th